CRNN Base Structure (TODO Training and Accuracy)

In [13]:
# Import required libraries
import numpy as np
import torch
import torch.nn as nn

In [14]:
# Base structure of CRNN
class CRNN(nn.Module):
    def __init__(self):
        super(CRNN, self).__init__()
        self.features = nn.Sequential (
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3,3)),
            nn.BatchNorm2d(64),
            nn.ELU(),
            nn.MaxPool2d((2,2)),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3)),
            nn.BatchNorm2d(128),
            nn.ELU(),
            nn.MaxPool2d((2, 2)),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3)),
            nn.BatchNorm2d(256),
            nn.ELU(),
            nn.MaxPool2d((2, 2)),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3)),
            nn.BatchNorm2d(256),
            nn.ELU(),
            nn.MaxPool2d((2, 2)),
        )

        final_conv_height = 6
        final_conv_width = 5
        gru_input_size = 256 * final_conv_width

        totalInstruments = 3

        self.gru = nn.GRU(
            input_size=768, 
            hidden_size=256, 
            batch_first=True
        )
        # out_features is technically 18 in paper, but realistically 3 at the beginning
        self.fc = nn.Linear(in_features=256, out_features=totalInstruments)
        
    def forward(self, input):
        # Pass input through convolutional layers
        conv_output = self.features(input)
        
        # Get the batch size (B), and the number of feature maps (C), height (H), and width (W)
        B, C, H, W = conv_output.size()

        # Reshape the output to treat the height 'H' as the sequence and combine the channels
        # and width 'W' as features, which should have been calculated as 256*5
        # Here we assume each timestep corresponds to a row in the feature maps
        x = conv_output.view(B, H, C * W)  # Shape: (B, H, C*W)

        # Pass the reshaped conv_output to the GRU
        gru_output, _ = self.gru(x)

        # Usually, you'd take just the last time step
        last_time_step_output = gru_output[:, -1, :]

        # Pass that through your fully connected layer
        output = self.fc(last_time_step_output)

        return output
    
    def save_model(self, path: str):
        torch.save(self.state_dict(), path)

    def load_model(*parameters, path: str):
        loaded_model = CRNN(parameters)
        loaded_model.load_state_dict(torch.load(path))
        return loaded_model

In [15]:
# Future stuff for training and accuracy of CNN
import torch.optim as optim

model = CRNN()
lossAlg = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.0001, momentum=0.9)

In [23]:
from torch.utils.data import DataLoader, Dataset, random_split
import logging, sys
import os
import h5py
import matplotlib.pyplot as plt

sys.path.append('/Users/adarshbharathwaj/Desktop/eng100/project3/ENGR_100_Project_3/src')
from utils import *



In [75]:
class FrameDataset(Dataset):
    def __init__(self, path):
        self.file_path = path
        self.file = h5py.File(self.file_path, 'r')
        self.features = self.file['features']
        self.labels = self.file['labels']

    def __len__(self):
        return int(self.file['overall_metadata'][2])

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]

        C, H, W = 1, 96, 87
        data_reshaped = feature.reshape(C, H, W)
        data_tensor = torch.tensor(data_reshaped, dtype=torch.float32)

        return data_tensor, label
    
    def close(self):
        self.file.close()



In [76]:
# Training the model

def train_model(
        model, train_dataloader, validation_dataloader, criterion, optimizer, epochs = 1
) -> Tuple[list, list]:
    train_accuracies = []
    validation_accuracies = []

    for epoch in range(epochs):
        model.train()

        total_loss = 0
        correct_predictions = 0
        total_predictions = 0

        for data, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            predicted = torch.sigmoid(outputs) > 0.5
            correct_predictions += (predicted == labels).float().sum()
            total_predictions += torch.numel(labels)

        train_accuracy = correct_predictions / total_predictions
        train_accuracies.append(train_accuracy.item())

        model.eval()
        with torch.no_grad():
            correct_predictions = 0
            total_predictions = 0
            for data, labels in validation_dataloader:
                outputs = model(data)
                predicted = torch.sigmoid(outputs) > 0.5
                correct_predictions = (predicted == labels).float().sum()
                total_predictions += torch.numel(labels)

            validation_accuracy = correct_predictions / total_predictions
            validation_accuracies.append(validation_accuracy)
    
    return train_accuracies, validation_accuracies

def plot_accuracy(
        train_accuracies:list, validation_accuracies:list, epoch_count: int
):
    epochs = range(1, epoch_count+1)
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_accuracies, label="Training Accuracy")
    plt.plot(epochs, validation_accuracies, label="Validation Accuracy")
    plt.title("Training and Validation Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

def test_model(model, test_dataloader):
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        correct_predictions = 0
        total_predictions = 0
        for data, labels in test_dataloader:
            outputs = model(data)
            predicted = torch.sigmoid(outputs) > 0.5
            correct_predictions += (predicted == labels).float().sum()
            total_predictions += torch.numel(labels)

Epoch [1/20], Loss: 0.5437
Epoch [2/20], Loss: 0.5294
Epoch [3/20], Loss: 0.4976
Epoch [4/20], Loss: 0.5224
Epoch [5/20], Loss: 0.4577
Epoch [6/20], Loss: 0.5226
Epoch [7/20], Loss: 0.4721


KeyboardInterrupt: 